In [117]:
import torch
import torchvision
import cv2
import numpy as np
from sort.sort import *
from deep_sort_realtime.deepsort_tracker import DeepSort

# YoloV5 Network and DeepSort

In [118]:
# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /home/ak232003/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-20 Python-3.9.13 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 2070 with Max-Q Design, 7982MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [119]:
object_tracker = DeepSort(max_iou_distance=1, max_age=200,nms_max_overlap = 5)

In [120]:
img = cv2.imread('cars.jpg')


In [121]:
results = model(img)


In [122]:
car_results = results.pred[0][results.pred[0][:, 5] == 2]


In [123]:
# Extract bounding boxes and confidences
boxes = car_results[:, :4].cpu().numpy()
confidences = car_results[:, 4].cpu().numpy()
class_label = car_results[:, 5].cpu().numpy()
# Draw bounding boxes around cars with confidence > 0.5
for box, confidence in zip(boxes, confidences):
    if confidence > 0.5:
        x1, y1, x2, y2 = box.astype(int)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Display image
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is n

In [132]:
boxes

array([[     370.95,      281.28,      502.87,      402.65],
       [     363.45,      113.66,      462.88,      194.07],
       [     194.29,      128.67,       290.5,      227.25],
       [      65.24,      76.512,      147.33,      146.56],
       [     531.52,      288.31,      620.47,       406.4],
       [     188.45,      66.085,      262.11,      132.07],
       [     304.24,      31.805,      371.04,      97.579],
       [     53.077,      19.183,      118.03,      81.227],
       [     268.24,      62.411,      357.13,       145.8],
       [     6.3308,      253.49,       153.1,      409.48],
       [     208.67,      260.73,      326.06,      392.78],
       [     309.87,       212.2,      433.79,       316.2],
       [     84.015,      162.04,      176.62,      269.13],
       [     176.49,      23.367,      247.38,       87.82],
       [     259.01,      2.9552,      326.68,      61.818],
       [     23.743,           0,      85.739,      42.381],
       [          0,    

In [125]:
def detect_cars(img, model, threshold = 0.5):
    detections = []
    pred = model(img)
    car_pred = pred.pred[0][pred.pred[0][:, 5] == 2]
    boxes = car_pred[:, :4].cpu().numpy()
    confidences = car_pred[:, 4].cpu().numpy()
    class_label = car_pred[:, 5].cpu().numpy()

    for i in range(len(class_label)):
        if(confidences[i] > threshold):
            cv2.rectangle(img, (int(boxes[i][0]), int(boxes[i][1])), (int(boxes[i][2]), int(boxes[i][3])), (0, 255, 0), 2)
            cv2.putText(img, "Car", (int(boxes[i][0])-10, int(boxes[i][1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            detections.append((boxes[i], confidences[i]))
    
    return detections

In [126]:
def update_track(tracks, count, track_ids):
    for t in tracks:
        if not t.is_confirmed():
            continue
        t_id = t.track_id
        if(t_id not in track_ids):
            count += 1
            track_ids.append(t_id)
        
        ltrb = t.to_ltrb()
        bbox = ltrb

    return track_ids, count

In [127]:
cap = cv2.VideoCapture('./mp4/Test.mp4')

In [128]:
track_ids = []
count = 0 

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detections = detect_cars(frame, model, 0.5)
    tracks = object_tracker.update_tracks(detections, frame=frame)
    track_ids, count = update_track(tracks, count, track_ids)
    cv2.putText(frame, f'Count: {int(count)}', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

    # Display result
    cv2.imshow('result', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and destroy windows
cap.release()
cv2.destroyAllWindows()
    

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is not the object's thread (0x7a26d900).
Cannot move to target thread (0x43dd7500)

QObject::moveToThread: Current thread (0x43dd7500) is n